# Kaggle Challenge - Exploring Mental Health

## Problem Statement

The goal of this project is to develop a machine learning model that can predict the likelihood of depression among individuals based on various features such as demographic information, academic and work-related pressures, lifestyle habits, and family history of mental illness. The dataset used for this project is sourced from a Kaggle challenge and contains multiple features that are indicative of an individual's mental health status.

### Objectives

1. **Data Preprocessing**: Clean and preprocess the dataset to make it suitable for training machine learning models. This includes handling missing values, encoding categorical variables, and normalizing numerical features.
2. **Model Development**: Develop a neural network model using PyTorch to predict the likelihood of depression. The model will be trained on a subset of the data and validated on another subset to evaluate its performance.
3. **Model Evaluation**: Evaluate the model's performance using appropriate metrics such as accuracy, loss, and validation accuracy. Fine-tune the model to improve its performance.
4. **Prediction**: Use the trained model to make predictions on a separate test dataset and save the results for further analysis.

### Dataset

The dataset contains the following features:

- **Gender**: Gender of the individual (encoded as integers).
- **Age**: Age of the individual.
- **City**: City of residence (encoded as integers).
- **Working Professional or Student**: Whether the individual is a working professional or a student (encoded as integers).
- **Profession**: Profession of the individual (encoded as integers).
- **Academic Pressure**: Level of academic pressure experienced by the individual.
- **Work Pressure**: Level of work pressure experienced by the individual.
- **CGPA**: Cumulative Grade Point Average (for students).
- **Study Satisfaction**: Satisfaction level with studies.
- **Job Satisfaction**: Satisfaction level with job.
- **Sleep Duration**: Average sleep duration.
- **Dietary Habits**: Dietary habits (encoded as integers).
- **Degree**: Highest degree obtained (encoded as integers).
- **Have you ever had suicidal thoughts?**: Whether the individual has ever had suicidal thoughts (encoded as integers).
- **Work/Study Hours**: Average number of work or study hours per day.
- **Financial Stress**: Level of financial stress experienced by the individual.
- **Family History of Mental Illness**: Whether there is a family history of mental illness (encoded as integers).
- **Depression**: Target variable indicating whether the individual is experiencing depression (binary).

### Expected Outcome

By the end of this project, we aim to have a well-trained neural network model that can accurately predict the likelihood of depression based on the provided features. The model's predictions will be saved and can be used for further analysis or interventions to support mental health initiatives.

### Import Libraries

In [108]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

### Load Dataset

In [109]:
# Load the CSV file into a DataFrame
df = pd.read_csv('../data/train.csv')

# Display the first few rows of the DataFrame
df.head()

,id,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,0,Aaradhya,Female,49.0,Ludhiana,Working Professional,Chef,NaN,5.0,NaN,NaN,2.0,More than 8 hours,Healthy,BHM,No,1.0,2.0,No,0
1,1,Vivan,Male,26.0,Varanasi,Working Professional,Teacher,NaN,4.0,NaN,NaN,3.0,Less than 5 hours,Unhealthy,LLB,Yes,7.0,3.0,No,1
2,2,Yuvraj,Male,33.0,Visakhapatnam,Student,NaN,5.0,NaN,8.97,2.0,NaN,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1
3,3,Yuvraj,Male,22.0,Mumbai,Working Professional,Teacher,NaN,5.0,NaN,NaN,1.0,Less than 5 hours,Moderate,BBA,Yes,10.0,1.0,Yes,1
4,4,Rhea,Female,30.0,Kanpur,Working Professional,Business Analyst,NaN,1.0,NaN,NaN,1.0,5-6 hours,Unhealthy,BBA,Yes,9.0,4.0,Yes,0


## Clean the data

In [110]:
# Remove the 'Name' column
df_cleaned = df.drop(columns=['Name', 'id'])

# Get the depression column and save it as a separate DataFrame
depression = df_cleaned['Depression']

# Remove the depression column from the original DataFrame
df_cleaned = df_cleaned.drop(columns=['Depression'])

# Assign a unique value for each column
for column in df_cleaned.columns:
    df_cleaned[column] = pd.factorize(df_cleaned[column])[0]

df_cleaned.head()

,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness
0,0,0,0,0,0,-1,0,-1,-1,0,0,0,0,0,0,0,0
1,1,1,1,0,1,-1,1,-1,-1,1,1,1,1,1,1,1,0
2,1,2,2,1,-1,0,-1,0,0,-1,2,0,2,1,2,2,0
3,1,3,3,0,1,-1,0,-1,-1,2,1,2,3,1,3,2,1
4,0,4,4,0,2,-1,2,-1,-1,2,2,1,3,1,4,3,1


In [111]:
print(df_cleaned.shape, depression.shape)

(140700, 17) (140700,)


In [112]:
X_train, X_test, y_train, y_test = train_test_split(
    df_cleaned, depression,
    test_size=0.2, random_state=42
)
print(X_train.shape, y_train.shape)

(112560, 17) (112560,)


In [113]:
# Convert to tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1)  # Reshape to (batch_size, 1)

X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).unsqueeze(1)  # Reshape to (batch_size, 1)


In [114]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [115]:
class DepressionModel(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(DepressionModel, self).__init__()
        # Define a fully connected neural network with 3 layers
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 128),  # First layer with 128 neurons
            nn.BatchNorm1d(128),
            nn.ReLU(),  # Activation function
            nn.Linear(128, hidden_dim),  # Second layer with hidden_dim neurons
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2),  # Dropout for regularization
            nn.Linear(hidden_dim, 64),  # Third layer with 64 neurons
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Linear(64, 32), # Fourth layer with 32 neurons
            nn.ReLU(),
            nn.Linear(32,1), # Output layer with 1 neuron
            nn.Sigmoid()  # Sigmoid activation for binary classification
        )

    def forward(self, x):
        # Forward pass through the network
        return self.fc(x)

### Initialize the Model, Loss Function, and Optimizer

In this step, we initialize the model, define the loss function, and set up the optimizer.

- **Model Initialization**: We create an instance of the `DepressionModel` class, passing the number of input features as the argument.
- **Loss Function**: We use Binary Cross-Entropy Loss (`BCELoss`) which is suitable for binary classification tasks.
- **Optimizer**: We use the Adam optimizer with a learning rate of 0.001 to update the model's parameters during training.


In [116]:
model = DepressionModel(input_dim=df_cleaned.shape[1], hidden_dim=512)
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

### Early Stopping

The following logic checks whether there is an increase in accuracy using a 'patience' and 'threshold' parameter and conditionally stops training

In [117]:
class EarlyStopping:
    def __init__(self, patience=5, thresh=0.001):
        """
        Args:
            patience (int, optional): Number of epochs to wait before stopping training if there is no increase in accuracy. Defaults to 5.
            thresh (float, optional): Value of expected accuracy increase. Defaults to 0.001.
        """
        self.patience=patience
        self.thresh=thresh
        self.best_acc=00.0
        self.counte=0

    def check(self, current_acc):
        """
        Check if early stopping should be triggered

        Args:
            current_acc (float): current validation accuracy that will be checked against best accuracy 
        """
        if self.thresh < current_acc - self.best_acc: 
            self.best_acc = current_acc
            self.counter=0
        else:
            self.counter+=1
        
        return self.counter >= self.patience
    

### Training the Model

In this step, we train the model using the training data and evaluate it on the validation data. The training process involves the following steps:

1. **Model Training**:
    In this step, we train the model using the training data and evaluate it on the validation data. The training process involves several key steps. First, we set the model to training mode and iterate over the training data in batches using `train_loader`. For each batch, we zero the gradients using `optimizer.zero_grad()`, perform a forward pass to compute the model's predictions, compute the loss using the criterion, perform a backward pass to compute the gradients, and finally update the model's weights using `optimizer.step()`.


2. **Model Evaluation** (optional):
    We evaluate the model by setting it to evaluation mode and iterate over the validation data in batches using `val_loader`. For each batch, we compute the model's predictions, accumulate the validation loss, and compute the number of correct predictions.

3. **Compute Accuracy**:
    Finally, we compute the accuracy of the model on the validation data. This involves calculating the accuracy based on the model's performance on the validation dataset.


In [118]:
num_epochs = 100  # Number of epochs for training
early_stopping = EarlyStopping(patience=5, thresh=0.001)

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    for inputs, labels in train_loader:
        optimizer.zero_grad()  # Zero the gradients

        outputs = model(inputs)  # Forward pass

        loss = criterion(outputs, labels)  # Compute the loss

        loss.backward()  # Backward pass

        optimizer.step()  # Update the weights

    # Validation (optional)
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():  # Disable gradient calculation
        val_loss = 0
        correct = 0
        total = 0
        for inputs, labels in val_loader:
            outputs = model(inputs)  # Forward pass
            val_loss += criterion(outputs, labels).item()  # Compute the validation loss
            predicted = (outputs > 0.5).float()  # Apply threshold for binary classification
            total += labels.size(0)  # Total number of samples
            correct += (predicted == labels).sum().item()  # Number of correct predictions

    accuracy = 100 * correct / total  # Compute accuracy
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Validation Loss: {val_loss / len(val_loader):.4f}, Accuracy: {accuracy:.4f}%")

    
    if early_stopping.check(accuracy):
        print(f"Early Stopping at epoch {epoch+1}, Best Accuracy: {early_stopping.best_acc}")
        break

Epoch [1/100], Loss: 0.2804, Validation Loss: 0.2512, Accuracy: 90.1173%
Epoch [2/100], Loss: 0.1617, Validation Loss: 0.2506, Accuracy: 90.0249%
Epoch [3/100], Loss: 0.2487, Validation Loss: 0.2439, Accuracy: 90.1599%
Epoch [4/100], Loss: 0.2085, Validation Loss: 0.2407, Accuracy: 90.3802%
Epoch [5/100], Loss: 0.4715, Validation Loss: 0.2419, Accuracy: 90.4087%
Epoch [6/100], Loss: 0.1489, Validation Loss: 0.2375, Accuracy: 90.1173%
Epoch [7/100], Loss: 0.1611, Validation Loss: 0.2311, Accuracy: 90.7249%
Epoch [8/100], Loss: 0.1684, Validation Loss: 0.2255, Accuracy: 91.0377%
Epoch [9/100], Loss: 0.1453, Validation Loss: 0.2251, Accuracy: 91.0839%
Epoch [10/100], Loss: 0.3910, Validation Loss: 0.2238, Accuracy: 90.9453%
Epoch [11/100], Loss: 0.0691, Validation Loss: 0.2226, Accuracy: 90.9986%
Epoch [12/100], Loss: 0.2441, Validation Loss: 0.2213, Accuracy: 91.2864%
Epoch [13/100], Loss: 0.5398, Validation Loss: 0.2246, Accuracy: 91.0483%
Epoch [14/100], Loss: 0.2649, Validation Loss: 

### Save the Model

After training the model, it is important to save it so that it can be used later without retraining. The model is saved using the `torch.save` function, which saves the model's state dictionary to a file. This allows us to load the model later and use it for making predictions on new data.

In [119]:
torch.save(model.state_dict(), '../models/trained.pth')
print("Model saved to '../models/trained.pth'")

Model saved to '../models/trained.pth'


In [120]:
# 8. Load and Preprocess the Test Data
test_df = pd.read_csv('../data/test.csv')

# Remove unnecessary columns
test_cleaned = test_df.drop(columns=['Name', 'id'])

# Factorize categorical columns (apply the same transformations as training data)
for column in test_cleaned.columns:
    test_cleaned[column] = pd.factorize(test_cleaned[column])[0]

# Convert to tensors
X_test_tensor = torch.tensor(test_cleaned.values, dtype=torch.float32)

# 9. Make Predictions on the Test Set
test_loader = DataLoader(X_test_tensor, batch_size=32, shuffle=False)

model.eval()
predictions = []
with torch.no_grad():
    for batch in test_loader:
        outputs = model(batch)
        preds = (outputs > 0.5).float()  # Apply threshold for binary classification
        predictions.extend(preds.cpu().numpy())

# Save predictions to a CSV file
predictions = pd.DataFrame(predictions, columns=['Predicted_Depression'])
predictions.to_csv('../data/test_predictions.csv', index=False)

print("Predictions saved to '../data/test_predictions.csv'")

Predictions saved to '../data/test_predictions.csv'
